In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd '/content/drive/MyDrive/Final_Model/stylegan2-swapper'

In [ ]:
CODE_DIR = 'StyleID'

!git clone https://github.com/minha12/StyleID.git
!wget https://github.com/ninja-build/ninja/releases/download/v1.8.2/ninja-linux.zip
!sudo unzip ninja-linux.zip -d /usr/local/bin/
!sudo update-alternatives --install /usr/bin/ninja ninja /usr/local/bin/ninja 1 --force

# os.chdir(f'./{CODE_DIR}')

In [ ]:
!pip install mtcnn

In [ ]:
from StyleID.mtcnn import mtcnn
import numpy as np

def detect_face(img):
  # Convert PIL Image to NumPy array
  img_array = np.asarray(img)
  # Step 1: Instantiate MTCNN
  face_recongniser = MTCNN()
  # Step 3: Face Detection
  faces = face_recongniser.detect_faces(img_array)
  return faces

  /content/drive/MyDrive/Final_Model/stylegan2-swapper/StyleID/models/mtcnn/mtcnn.py

In [ ]:
import imageio
import requests
import bz2
from PIL import Image
import torch
import torchvision.transforms as transforms
import dlib
from pix2pixHD.data.base_dataset import __scale_width
from pix2pixHD.models.networks import define_G
import pix2pixHD.util.util as util
from aligner import align_face

import matplotlib.pyplot as plt
%matplotlib inline

from google.colab import output
from IPython.display import HTML
import base64
import imageio

In [ ]:
from PIL import Image
import IPython.display as display
import io

img_url = '/content/drive/MyDrive/Final_Model/Test_F2.jpg'
file_name = 'input_image.jpg'
# Open the image using PIL
with open(img_url, 'rb') as f:
    img = Image.open(io.BytesIO(f.read()))
# display.display(img)

In [ ]:
faces = detect_face(img)
if faces is not None:
  imageio.v2.imwrite(file_name, imageio.v2.imread(img_url))
  # print("Found Faces")

**Face Alignment**

In [ ]:
def unpack_bz2(src_path):
    data = bz2.BZ2File(src_path).read()
    dst_path = src_path[:-4]
    with open(dst_path, 'wb') as fp:
        fp.write(data)
    return dst_path

def download(url, file_name):
    with open(file_name, "wb") as file:
        response = requests.get(url)
        file.write(response.content)

In [ ]:
# Pre-trained model for facial landmark detection

import bz2
import shutil

# Define the paths
shape_model_drive_path = '/content/drive/MyDrive/Final_Model/shape_predictor_68_face_landmarks.dat.bz2'
shape_model_extracted_path = '/content/drive/MyDrive/Final_Model/shape_predictor_68_face_landmarks.dat'

# Extract the contents in Google Drive
with open(shape_model_extracted_path, 'wb') as new_file, bz2.BZ2File(shape_model_drive_path, 'rb') as file:
    shutil.copyfileobj(file, new_file)

# Load the model
shape_predictor = dlib.shape_predictor(shape_model_extracted_path)

In [ ]:
# aligned_img = align_face(file_name, shape_predictor)[0]

**Prepare pix2pixHD model**

In [ ]:
def get_eval_transform(loadSize=512):
    transform_list = []
    transform_list.append(transforms.Lambda(lambda img: __scale_width(img,
                                                                      loadSize,
                                                                      Image.BICUBIC)))
    transform_list += [transforms.ToTensor()]
    transform_list += [transforms.Normalize((0.5, 0.5, 0.5),
                                            (0.5, 0.5, 0.5))]
    return transforms.Compose(transform_list)

transform = get_eval_transform()

In [ ]:
# To male:
!gdown --id 1-6J1CYLsIysk38X9DNN23lIcnvOr8aYh # https://drive.google.com/file/d/1-6J1CYLsIysk38X9DNN23lIcnvOr8aYh/view?usp=sharing

# To female:
!gdown --id 1frJERJr0WM_R38LnSFQ6XjGQtcXnLco1 # https://drive.google.com/file/d/1frJERJr0WM_R38LnSFQ6XjGQtcXnLco1/view?usp=sharing

In [ ]:
config_G = {
    'input_nc': 3,
    'output_nc': 3,
    'ngf': 64,
    'netG': 'global',
    'n_downsample_global': 4,
    'n_blocks_global': 9,
    'n_local_enhancers': 1,
    'norm': 'instance',
}

weights_path = '/content/drive/MyDrive/Final_Model/to_male_net_G.pth'  # to_female_net_G.pth

model = define_G(**config_G)
pretrained_dict = torch.load(weights_path)
model.load_state_dict(pretrained_dict)
model.cuda();

**Use Model**

In [ ]:
img = transform(aligned_img).unsqueeze(0)

with torch.no_grad():
    out = model(img.cuda())

out = util.tensor2im(out.data[0])

In [ ]:
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.imshow(aligned_img)
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(out)
plt.axis('off')

plt.tight_layout();

**Save the output**

In [ ]:
imageio.imsave('result.jpg', out)

In [ ]:
import torch
import torchvision.transforms as transforms
from PIL import Image
from torchvision import models

# Load the pre-trained ResNet-18 model
model = models.resnet18(pretrained=True)

# Freeze the model parameters
for param in model.parameters():
    param.requires_grad = False

# Modify the last fully connected layer for gender classification
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 2)  # 2 classes: male and female

# Load the weights from a checkpoint
checkpoint_path = '/content/drive/MyDrive/Final_Model/face_gender_classification_transfer_learning_with_ResNet18.pth'
checkpoint = torch.load(checkpoint_path)
model.load_state_dict(checkpoint)


# Load the image and perform transformations
def load_image(image_path):
    image = Image.open(image_path).convert('RGB')
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    return preprocess(image).unsqueeze(0)

# Define class labels
class_labels = {0: 'Male', 1: 'Female'}

# Function to predict gender
def predict_gender(image_path):
    model.eval()
    image_tensor = load_image(image_path)
    outputs = model(image_tensor)
    _, predicted = torch.max(outputs, 1)
    predicted_label = class_labels[predicted.item()]
    return predicted_label

# Example usage: path to the image you want to classify
result = predict_gender(img_url)
print(f"Predicted gender: {result}")

In [ ]:
checkpoint = torch.load(checkpoint_path)
print(checkpoint.keys())

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Convert the PyTorch tensor to a NumPy array
img_np = img.squeeze().permute(1, 2, 0).cpu().numpy()
# print(img_np.shape)
# Reshape the NumPy array to match the shape of the PyTorch tensor
img_np_flat = img_np.flatten()
# print(img_np_flat.shape)
# Reshape the NumPy array to a 1D array
out_flat = out.flatten()
# print(out_flat.shape)
# Reshape the NumPy array to a 2D array (1 sample, -1 features)
img_np_flat = img_np_flat.reshape(1, -1)
out_flat = out_flat.reshape(1, -1)
# print(img_np_flat.shape)
# print(out_flat.shape)

# Calculate cosine similarity using sklearn
cosine_sim = cosine_similarity(img_np_flat, out_flat)

# Print the cosine similarity
print("Cosine Similarity:", cosine_sim[0, 0])

# Convert cosine similarity to percentage
percentage_similarity = ((cosine_sim[0, 0] + 1) / 2) * 100

# Print the percentage similarity
print(f"Similarity Percentage: {percentage_similarity:.2f}%")
# print(img.shape)
# print(out.shape)

# input_tensor = img.to(device)
# output_tensor = out.to(device)

In [ ]:
import torch
import numpy as np

# Convert the NumPy array to a PyTorch tensor
output_tensor = torch.from_numpy(out).permute(2, 0, 1).unsqueeze(0).float()

# Print the shape of the resulting tensor
# print("Tensor Shape:", tensor.shape)
# print(len(tensor))
# Flatten the tensors to 1D vectors (if needed)
tensor1_flat = img.view(1, -1)
tensor2_flat = output_tensor.view(1, -1)


# define a method to measure cosine similarity
cos = torch.nn.CosineSimilarity(dim=1)
output = cos(tensor1_flat, tensor2_flat)

# display the output tensor
print("Cosine Similarity:",output)

# Assuming 'output' contains the cosine similarity value
percentage_similarity = ((output.item() + 1) / 2) * 100
print(f"Similarity Percentage: {percentage_similarity:.2f}%")

In [ ]:
# using torch
def find_cosine_similarity(input_tensor, generated_image):
  # Convert the NumPy array to a PyTorch tensor
  output_tensor = torch.from_numpy(generated_image).permute(2, 0, 1).unsqueeze(0).float()
  # Flatten the tensors to 1D vectors (if needed)
  tensor1_flat = input_tensor.view(1, -1)
  tensor2_flat = output_tensor.view(1, -1)

  # define a method to measure cosine similarity
  cos = torch.nn.CosineSimilarity(dim=1)
  output = cos(tensor1_flat, tensor2_flat)

  # display the output tensor
  return output

def get_similarity_percentage_using_torch(input_tensor, generated_image):
  cos = find_cosine_similarity(input_tensor, generated_image)
  percentage= ((cos.item() + 1) / 2) * 100
  return percentage

print(find_cosine_similarity(img, out))
print(get_similarity_percentage_using_torch(img, out))




In [ ]:
import os
from PIL import Image
import torchvision.transforms as transforms
import time


model.eval()

# Path to your dataset folder in Google Drive
dataset_folder = '/content/drive/MyDrive/Final_Model/Normal_Image_Testing'

sum_of_percentage = 0
num_of_files = 0

# Start the timer
start_time = time.time()
# Iterate through all files in the folder
for filename in os.listdir(dataset_folder):
    if filename.endswith('.jpg') or filename.endswith('.png'):  # Check if it's an image file
        # Load the image using PIL from the dataset folder
        aligned_img = align_face(os.path.join(dataset_folder, filename), shape_predictor)[0]
        img = transform(aligned_img).unsqueeze(0)
        with torch.no_grad():
            output = model(img.cuda())
        out = util.tensor2im(output.data[0])

        sum_of_percentage += get_similarity_percentage_using_torch(img, out)
        num_of_files += 1

# Stop the timer
end_time = time.time()

# Calculate the time taken
elapsed_time = end_time - start_time
elapsed_minutes = int(elapsed_time_seconds // 60)
elapsed_seconds = int(elapsed_time_seconds % 60)

average = sum_of_percentage / num_of_files
print(f"Average Similarity Percentage: {average:.2f}%")
print(f"Time taken: {elapsed_minutes} minutes and {elapsed_seconds} seconds of {num_of_files} files")



In [ ]:
import os
from PIL import Image
import torchvision.transforms as transforms
import time


model.eval()

# Path to your dataset folder in Google Drive
dataset_folder = '/content/drive/MyDrive/Final_Model/Normal_Image_Testing'

sum_of_percentage = 0
num_of_files = 0

# Start the timer
start_time = time.time()
# Iterate through all files in the folder
for filename in os.listdir(dataset_folder):
    if filename.endswith('.jpg') or filename.endswith('.png'):  # Check if it's an image file
        # Load the image using PIL from the dataset folder
        input_image = Image.open(os.path.join(dataset_folder, filename))
        # Detect faces using the detect_faces method
        bounding_boxes, landmarks = face_recongniser.detect_faces(input_image)

        aligned_img = align_face(os.path.join(dataset_folder, filename), shape_predictor)[0]
        img = transform(aligned_img).unsqueeze(0)
        with torch.no_grad():
            output = model(img.cuda())
        out = util.tensor2im(output.data[0])

        sum_of_percentage += get_similarity_percentage_using_torch(img, out)
        num_of_files += 1

# Stop the timer
end_time = time.time()

# Calculate the time taken
elapsed_time = end_time - start_time
elapsed_minutes = int(elapsed_time_seconds // 60)
elapsed_seconds = int(elapsed_time_seconds % 60)

average = sum_of_percentage / num_of_files
print(f"Average Similarity Percentage: {average:.2f}%")
print(f"Time taken: {elapsed_minutes} minutes and {elapsed_seconds} seconds of {num_of_files} files")

